In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import json

#Linkar
links = ["https://understat.com/league/EPL/2024", "https://understat.com/league/EPL/2023", "https://understat.com/league/EPL/2022",
         "https://understat.com/league/EPL/2021", "https://understat.com/league/EPL/2020",
         "https://understat.com/league/EPL/2019", "https://understat.com/league/EPL/2018",
         "https://understat.com/league/EPL/2017"]

all_data = {}

for link in links:
    try:
        res = requests.get(link)
        res.raise_for_status()

        soup = bs(res.content, 'lxml')

        scripts = soup.find_all('script')

        strings = scripts[1].string

        #hreinsa
        ind_start = strings.index("('") + 2
        ind_end = strings.index("')")
        json_data = strings[ind_start:ind_end]

        json_data = json_data.encode('utf8').decode('unicode_escape')

        data = json.loads(json_data)

        all_data[link] = data

    except Exception as e:
        print(f"An error occurred while processing {link}: {e}")

print(all_data)



{'https://understat.com/league/EPL/2024': [{'id': '26602', 'isResult': True, 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'goals': {'h': '1', 'a': '0'}, 'xG': {'h': '2.04268', 'a': '0.418711'}, 'datetime': '2024-08-16 19:00:00', 'forecast': {'w': '0.8069', 'd': '0.1489', 'l': '0.0442'}}, {'id': '26603', 'isResult': True, 'h': {'id': '285', 'title': 'Ipswich', 'short_title': 'IPS'}, 'a': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'goals': {'h': '0', 'a': '2'}, 'xG': {'h': '0.342601', 'a': '3.92906'}, 'datetime': '2024-08-17 11:30:00', 'forecast': {'w': '0.0026', 'd': '0.018', 'l': '0.9794'}}, {'id': '26604', 'isResult': True, 'h': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'a': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'goals': {'h': '2', 'a': '0'}, 'xG': {'h': '1.6283', 'a': '0.575835'}, 'datetime': '2024-08-17 14:00:00', 'forecast': {'w': '0

In [7]:
dataframes = []

for link, data in all_data.items():
    try:
        df = pd.DataFrame(data)
        dataframes.append(df)

    except Exception as e:
        print(f"An error occurred while converting data from {link} to DataFrame: {e}")

In [9]:
merged_df = pd.concat(dataframes, ignore_index=True)

In [10]:
match_ids = merged_df['id']

In [11]:
#nú getum við sótt advanced stats með öllum id's
import requests
import re
import json
import pandas as pd

all_matches = []

#loopum í gegn
for match_id in match_ids:
    try:
        #sækjum tölfræði fyrir hvert match-id
        response = requests.get(f'https://understat.com/match/{match_id}')

        #laga til
        match_info = re.search(r"match_info\s+=\s+JSON.parse\('([^']+)", response.text)
        decoded_string = bytes(match_info.groups()[0], 'utf-8').decode('unicode_escape')
        matchObj = json.loads(decoded_string)

        all_matches.append(matchObj)

    except Exception as e:
        print(f"An error occurred for match ID {match_id}: {e}")

df = pd.DataFrame(all_matches)


In [14]:
#vista
import pickle

pick_insert = open('drive/My Drive/Salka ML football/data/understat_epl.pickle','wb')
pickle.dump(df, pick_insert)
pick_insert.close()

In [15]:
df

,id,fid,h,a,date,league_id,season,h_goals,a_goals,team_h,...,h_l,league,h_shot,a_shot,h_shotOnTarget,a_shotOnTarget,h_deep,a_deep,a_ppda,h_ppda
0,26602,1821049,89,228,2024-08-16 19:00:00,1,2024,1,0,Manchester United,...,0.0442,EPL,14,10,5,2,7,3,10.8333,7.3793
1,26603,1821050,285,87,2024-08-17 11:30:00,1,2024,0,2,Ipswich,...,0.9794,EPL,7,19,2,5,2,13,8.7391,18.7778
2,26604,1821052,83,229,2024-08-17 14:00:00,1,2024,2,0,Arsenal,...,0.1162,EPL,18,9,6,3,14,2,10.8182,7.7692
3,26605,1821054,72,220,2024-08-17 14:00:00,1,2024,0,3,Everton,...,0.7884,EPL,9,10,1,5,4,5,7.9167,18.3333
4,26606,1821055,86,74,2024-08-17 14:00:00,1,2024,1,0,Newcastle United,...,0.7982,EPL,3,19,1,5,4,13,3.7895,16.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,7494,1190475,219,83,2018-05-13 15:00:00,1,2017,0,1,Huddersfield,...,0.6604,EPL,18,9,3,4,4,8,16.4167,9.6563
3036,7495,1190477,87,220,2018-05-13 15:00:00,1,2017,4,0,Liverpool,...,0.0013,EPL,22,2,11,1,19,2,49.0000,7.8571
3037,7496,1190479,89,90,2018-05-13 15:00:00,1,2017,1,0,Manchester United,...,0.3511,EPL,7,7,1,3,5,5,9.2593,18.7143
3038,7497,1190547,86,80,2018-05-13 15:00:00,1,2017,3,0,Newcastle United,...,0.0436,EPL,16,6,6,2,4,3,8.5652,10.3548


In [16]:
understat_epl = df[['date', 'team_h', 'team_a', 'h_goals', 'a_goals',
                   'h_xg', 'a_xg', 'h_deep', 'a_deep', 'h_ppda', 'a_ppda']]
understat_epl

,date,team_h,team_a,h_goals,a_goals,h_xg,a_xg,h_deep,a_deep,h_ppda,a_ppda
0,2024-08-16 19:00:00,Manchester United,Fulham,1,0,2.04268,0.418711,7,3,7.3793,10.8333
1,2024-08-17 11:30:00,Ipswich,Liverpool,0,2,0.342601,3.92906,2,13,18.7778,8.7391
2,2024-08-17 14:00:00,Arsenal,Wolverhampton Wanderers,2,0,1.6283,0.575835,14,2,7.7692,10.8182
3,2024-08-17 14:00:00,Everton,Brighton,0,3,0.405325,1.79083,4,5,18.3333,7.9167
4,2024-08-17 14:00:00,Newcastle United,Southampton,1,0,0.433489,1.95483,4,13,16.2500,3.7895
...,...,...,...,...,...,...,...,...,...,...,...
3035,2018-05-13 15:00:00,Huddersfield,Arsenal,0,1,1.31031,2.3322,4,8,9.6563,16.4167
3036,2018-05-13 15:00:00,Liverpool,Brighton,4,0,3.50981,0.0408165,19,2,7.8571,49.0000
3037,2018-05-13 15:00:00,Manchester United,Watford,1,0,0.45805,0.654288,5,5,18.7143,9.2593
3038,2018-05-13 15:00:00,Newcastle United,Chelsea,3,0,2.24197,0.843284,4,3,10.3548,8.5652


In [17]:
understat_epl['date'] = pd.to_datetime(understat_epl['date'], format='%Y-%m-%d %H:%M:%S')
understat_epl['date'] = understat_epl['date'].dt.strftime('%Y/%m/%d')

In [18]:
understat_epl_ready = understat_epl.sort_values(by='date')

In [19]:
understat_epl_ready

,date,team_h,team_a,h_goals,a_goals,h_xg,a_xg,h_deep,a_deep,h_ppda,a_ppda
2660,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455
2667,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444
2661,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143
2662,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000
2663,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824
...,...,...,...,...,...,...,...,...,...,...,...
371,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,17.8667
370,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,14.1176
379,2025/05/25,Fulham,Manchester City,0,2,2.21797,2.45923,5,16,22.1538,13.5455
372,2025/05/25,Manchester United,Aston Villa,2,0,2.96259,0.335222,24,5,5.1071,26.9091


In [20]:
understat_epl_ready.columns = ['Date','Home_Team','Away_Team','Home_Goals','Away_Goals','Home_xG',
                               'Away_xG', 'Home_Deep', 'Away_Deep',
                               'Home_ppda', 'Away_ppda']
understat_epl_ready

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda
2660,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455
2667,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444
2661,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143
2662,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000
2663,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824
...,...,...,...,...,...,...,...,...,...,...,...
371,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,17.8667
370,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,14.1176
379,2025/05/25,Fulham,Manchester City,0,2,2.21797,2.45923,5,16,22.1538,13.5455
372,2025/05/25,Manchester United,Aston Villa,2,0,2.96259,0.335222,24,5,5.1071,26.9091


In [21]:
import pickle

pick_insert = open('drive/My Drive/Salka ML football/data/understat_epl_ready.pickle','wb')
pickle.dump(understat_epl_ready, pick_insert)
pick_insert.close()